$\renewcommand{\vec}[1]{\mathbf{#1}}$


<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Американские горки

### Examples - Mechanics
<section class="post-meta">
By Jonas Tjemsland, Eilif Sommer Øyre and Jon Andreas Støvneng
</section>
Last edited: April 17th 2018 
___


## Вступление

В серии лабораторных занятий по курсу физики в НТНУ студенты изучают движение объектов по криволинейной дорожке. Из-за кривизны скорость и ускорение будут меняться. Студенты используют высокоскоростную камеру для записи движения и сравнения результатов с численным моделированием.

В этом блокноте мы смоделируем катящийся шар на некоторой произвольной дорожке в двух измерениях, используя второй закон Ньютона. Дорожка создается путем выполнения интерполяции кубического сплайна по набору точек. Для получения дополнительной информации см. [1].

Во-первых, мы импортируем необходимые библиотечные пакеты.

In [ ]:
import numpy as np
import scipy.interpolate as interp
import matplotlib.pyplot as plt
plt.style.use('bmh')    # Nicer looking plots

## Теория
#### Уравнение движения

Небольшой шарик, катится по изогнутой дорожке. Ось вращения проходит через центр масс. Пусть $m$ - масса, $r$ - радиус шара и $I_n$ - момент инерции. Силы, действующие на центр масс, описываются вторым законом Ньютона

\begin{equation}
\mathbf F = m\mathbf a.
\end{equation}


Силы, действующие на шар, можно разложить на компонент, параллельный дорожке, и компонент, перпендикулярный дорожке, как показано на рис.1. Пусть локальный наклон дорожки описывается углом $\theta(x)$. Гравитационная сила действует на центр масс вертикально вниз. Это равносильно компоненту, параллельному треку $mg\sin \theta(x)$, и нормальному компоненту $mg\cos \theta(x)$. Силы от дорожки к шару имеют нормальную составляющую $N$ (нормальная сила) и параллельную составляющую $f$ (трение).


<img src="images/rolling-forces.png">

*Рисунок 1: Цилиндр или сфера катится по кривой $y(x)$. Силы, действующие на объект, указаны на рисунке. В каждой точке $x$ наклон кривой определяется углом $\theta(x)$.* 


Из-за кривизны дорожки центр масс имеет центростремительное ускорение, нормальное к дорожке. Таким образом, общее ускорение определяется

\begin{equation}
\mathbf a = \dot v \mathbf{e}_\parallel + \frac{v^2}{R(x)} \mathbf{e}_\perp,
\end{equation}
    
где $R(x)$ - это *радиус кривизны*, а $v$ - скорость (вдоль трека). Таким образом, мы получаем два уравнения

\begin{equation}
mg \sin \theta(x)-f=m\dot v,
\label{eq:parallel}\quad(1)
\end{equation}
\begin{equation}
N-mg\cos\theta(x) = \frac{mv^2}{R(x)}.
\label{eq:normal}\quad(2)
\end{equation}

Первое уравнение описывает движение параллельно дорожке, в то время как второе уравнение дает, с $N\geq 0$, условие для того, когда мяч теряет контакт с поверхностью.

Выражение для трения найдено с помощью второго закона вращения Ньютона,

\begin{equation}
\tau = I_0\omega.
\end{equation}

Трение $f$ является единственной силой, которая не проходит через ось вращения, и, таким образом, является единственной силой, которая вносит вклад в общий крутящий момент $\tau$. Трение действует на объект на расстоянии $r$ от оси вращения (и под углом $\pi/2$), так что $fr = I_0 \omega$. Предполагается, что мяч катится без какого-либо скольжения. Используя условие качения $v=\omega r$, уравнение (1) можно записать в виде

\begin{equation}
\dot v = \frac{g\sin (\theta(x))}{1 + I_0/mr^2}.
\label{eq:vdot}\quad(*)
\end{equation}

Полное обсуждение динамики вращения можно найти, например, в [главах 9-10, 2] и [главе 6, 3].

#### Кривизна

Рассмотрим кривую $y(x)$ от $A$ до $B$, как показано на рис.2. Кривую от $A$ до $B$ с длиной дуги $\Delta s$ можно рассматривать как сектор малого круга с углом $\Delta \theta$. Окружность имеет центр в $C$ и радиус $R\approx R_A\approx R_B$. Таким образом, радиус кривизны становится $R=\Delta s/\Delta \theta$. *Кривизна*, в свою очередь, определяется как $\kappa= 1/R$.


<img src="images/rolling-curvature.png">
*Рисунок 2: Эскиз, используемый для описания кривизны и радиуса кривизны. (Взято из [1])* 


Рассмотрим теперь бесконечно малую дугу $\Delta s \to\text{d} s$. Из рисунка видно, что

\begin{equation}
\text{d}y/\text{d} x = \tan\theta,
\label{eq:theta}\quad(3)
\end{equation}

что дает

$$\frac{\text{d} \theta}{\text{d}x} = \frac{\text{d}}{\text{d}x}\arctan\left(\frac{\text{d}y}{\text{d}x}\right)=\frac{1}{1+\left(\text d y/\text dx\right)^2}\frac{\text d^2 y}{\text dx^2}.$$

Дифференциал $\text d s$ задается

$$(\text{d}s)^2 = (\text d x)^2 + (\text d y)^2 \quad \Longrightarrow \quad \text d s=\text d x\sqrt{1 + \left(\text d y/dx\right)^2}.$$

Таким образом, кривизна может быть записана как

\begin{equation}
\kappa =\frac{\text{d}\theta}{\text{d}s} = \frac{\text{d}^2 y/\text{d}x^2}{\left(1 +\left(\text d y/\text d x\right)^2\right)^{3/2}},
\end{equation}

и радиус кривизны становится

\begin{equation}
R(x) = \frac{\text{d}s}{\text{d}\theta} = \frac{\left(1 +\left(\text d y/\text d x\right)^2\right)^{3/2}}{\text{d}^2 y/\text{d}x^2}.
\label{eq:R}\quad(4)
\end{equation}

Обратите внимание, что $\kappa$ всегда конечна, пока $\text d y/\text d x$ непрерывен, в то время как $R\to \infty$, если $\text d^2 y/\text d x^2\to 0$. 

Кривизна более подробно обсуждается, например, в [chap. 11, 4] и [chap. 13, 5].

## Установка

Мы будем рассматривать твердую сферу (шар), которая имеет момент инерции $I_0= 2mr^2/5$. Момент инерции легко вычисляется для таких объектов, но это не будет обсуждаться здесь. См., например, [глава 9, 2] для общего обсуждения и [6] для списка некоторых типичных значений для $I_0$. Предположим, что шар имеет радиус $r=1\,\text{cm}$ и изготовлен из железа с плотностью $\rho=7850\,\text{kg/m}^3$ (плотность найдена в [7]). Масса шара в этом случае равна $m= 32,9\,\text{g}$, а момент инерции,таким образом, равен $I_0=13,2\,\mathrm{g\, cm^2}$.

In [ ]:
# Properties of the rolling object
r = 0.01                 # m      (radius)
rho = 7850               # kg/m^3 (density)
g = 9.81                 # m/s^2  (gravitational acceleration)

m = (4/3)*np.pi*r**3*rho # kg     (mass)
c = 2/5
I0 = c*m*r**2            # kg m^2 (moment of inertia)

<img src="images/roller-coaster.png">

*Рис. 3: Фактическая установка используемая в лаборатории показана слева. Справа показан эскиз. (Правое изображение взято из [1])*


Дорожка выполнена из пластикового стержня, который крепится к прочной раме на $N=8$ различных креплениях, как показано на рис. 3. Пусть $(x_i, y_i)$ - позиции опорных узлов. Позиции $x$ фиксированы и равномерно распределены по рамке. Будем считать, что установка имеет длину $L=1.4\,\text{м.}$, такую, что расстояние между узлами составляет $l = 20\,\text{см}$. Позиции $y$ могут меняться.

In [ ]:
# Properties of the frame
L = 1.4                   # m (length)
yi = [.5, .3, .25, .3,
     .35, .34, .25, .15]  # m (y-positions)

N = len(yi)               #   (# of mounts)
xi = np.linspace(0, L, N) # m (x-positions)

Трек может быть описан кубическим сплайном, который интерполирует $(x_i, y_i)$. Кубический сплайн состоит из набора кубических многочленов, которые имеют непрерывные первую и вторую производные в точках интерполяции. Смотрите наш блокнот про [Кубические сплайны](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/cubic_splines.ipynb) для общего обсуждения. Мы будем использовать функцию [`CubicSpline`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html#scipy.interpolate.CubicSpline) из `scipy.interpolate` для выполнения интерполяции.

In [ ]:
# Callable class for the track curve
get_y = interp.CubicSpline(xi, yi, bc_type="natural")

Как мы видели ранее, $\theta$ и $R$ зависят от $\text dy/\text dx$ и $\text d^2y/\text dx^2$. Класс `CubicSpline` имеет функцию, называемую `derivative` которая возвращает класс `PPoly`, который, по сути, является производной от сплайна. `PPoly` имеет также функцию `derivative`. Таким образом, мы можем легко вычислить $\theta(x)$ и $R(x)$, используя уравнения (3) и (4).

In [ ]:
get_dydx = get_y.derivative()      # Callable derivative of track
get_d2ydx2 = get_dydx.derivative() # Callable double derivative of track

def get_theta(x):
    """ Возвращает угол наклона дорожки. """
    return -np.arctan(get_dydx(x))
    
def get_R(x):
    """ Возвращает радиус кривизны. """
    return (1 + (get_dydx(x))**2)**1.5/get_d2ydx2(x)

def get_curvature(x):
    """ Возвращает кривизну (1/R). """
    return get_d2ydx2(x)/(1 + (get_dydx(x))**2)**1.5

Давайте построим дорожку!

In [ ]:
x = np.linspace(xi[0], xi[-1], 200)

# Create figure
fig, axarr = plt.subplots(3, 1, sharex=True, figsize=(12, 9), dpi=600)
fig.subplots_adjust(hspace=0.02)

# Axes 1:
axarr[0].plot(x, get_y(x), 'C0', label=r"$y(x)$")
axarr[0].plot(xi, yi, 'C1o', label="Mounts")
axarr[0].set_ylabel(r"$y(x)$, [m]", size='15')
#axarr[0].set_aspect('equal')

# Axes 2: 
axarr[1].plot(x, get_theta(x), 'C0')
axarr[1].set_ylabel(r"$\theta(x)$, [rad]", size='15')

# Axes 2: 
axarr[2].plot(x, get_curvature(x), 'C0')
axarr[2].set_ylabel(r"$\kappa(x)$, [1/m]", size='15')

plt.show()

Мы предполагаем, что потери механической энергии не происходит. Таким образом, если самая высокая точка трека равна $y(x=0)$, то мяч упадет вправо. Если это не так, мяч будет кататься взад и вперед. 

## Numerics

Мы начинаем с предположения, что мяч всегда находится в контакте с дорожкой. Уравнение (*) дает систему обыкновенных дифференциальных уравнений (ODE)

$$\frac{\text dv}{\text dt} = \frac{g\sin (\theta(t))}{1 + I_0/mr^2},$$

где $v$ - мгновенная скорость вдоль трассы. Позиция $x$, в свою очередь, задается

$$ \text dx = \text ds \cos(\theta)\: \Longrightarrow \: \frac{\text dx}{\text dt}  = v\cos(\theta).$$

In [ ]:
def get_vdot(theta):
    """ Возвращает производную по времени от (полной) скорости. """
    return g*np.sin(theta)/(1 + c)

def RHS(z):
    """ Оценивает правую часть двух связанных ОДУ.
    
    Parameters:
        z : array-like, len(2), float. [v, x]
            The first element is the velocity, the second is the x-position.
        
    Returns:
        array-like, len(2), float. [a, v]
        The first element is the time derivative of the velocity (acceleration),
        the second is the time derivative of the position (velocity).
    """
    # z = [x, v]
    # w = [vx, a]
    w = np.zeros(2)
    theta = get_theta(z[0])
    w[0] = z[1]*1/np.sqrt(1+np.tan(theta)**2)
    w[1] = get_vdot(theta)
    return w

$v(t)$ и $x(t)$ можно найти, применив метод решения ОДУ, такой как [Метод Эйлера](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/eulers_method.ipynb), [Метод Рунге-Кутты](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/runge_kutta_method.ipynb) или более продвинутый [адаптивный метод](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/adaptive_runge_kutta_methods.ipynb). Мы отсылаем вас к соответствующим модулям или к примеру, который решает ОДУ в [numfys.net](https://www.numfys.net). В этом блокноте мы будем использовать метод Рунге-Кутты 4-го порядка.

In [ ]:
def rk4step(f, y, h):
    """ Выполняет один шаг метода Рунге-Кутты 4-го порядка.
    
    Parameters:
        f : Callable function with one input parameter.
            The right hand side of the ODE. Note that the
            RHS is in our case not a function of time.
        y : array-like, float. Current position.
        h : float. Time step.
    """
    s1 = f(y)
    s2 = f(y + h*s1/2.0)
    s3 = f(y + h*s2/2.0)
    s4 = f(y + h*s3)
    
    return y + h/6.0*(s1 + 2.0*s2 + 2.0*s3 + s4)

Мы выбираем $x=0$ и $v=0$ в качестве начальных условий и временной шаг $\Delta t = 10^{-3}\, \text{s}$. Мы предположили, что шар не теряет никакой механической энергии. Один из методов проверки того, дает ли вычисление реалистичные результаты, состоит в том, чтобы проверить, является ли общая механическая энергия

\begin{equation}
E = \frac{1}{2}mv^2 + mgh + \frac{1}{2}I_0 \omega^2 = \frac{1}{2}m(1 + c)v^2 + mgh,
\end{equation}
постоянной.

In [ ]:
dt = 1e-3 # s
tstop = 5 # s. If the ball has not reached the end within 5 seconds, we stop.
x0 = 0    # m. Initial x-position
v0 = 0    # m/s. Inital velocity

def get_K(v):
    """ Returns the kinetic energy. """
    return .5*m*(1 + c)*v**2

def get_U(h):
    """ Returns the potential energy. """
    return m*g*h

Теперь все готово к тому, чтобы катить мяч по дорожке. 

In [ ]:
# Set initial values
x = [x0]        # x-position
v = [v0]        # velocity
h = get_y(x0)   # height
K = [get_K(v0)] # kinetic energy
U = [get_U(h)]  # potential energy

it = 0 # Iterator
itmax = tstop/dt # Maximum number og iterations
while x0 <= L and it < itmax:
    # Perform one step of the Runge-Kutta method
    [x0, v0] = rk4step(RHS, [x0, v0], dt)
    # Append different values to their arrays
    x = np.append(x, x0)
    v = np.append(v, v0)
    h = get_y(x0)
    K = np.append(K, get_K(v0))
    U = np.append(U, get_U(h))
    it += 1

print("Iterations: %i"%(it))
print("Final time: %.2f s"%(it*dt))
dE = (K[0] - K[-1] + U[0] - U[-1])/(K[0] + U[0])
print("Relative change in mechanical energy: %.2e"%(dE))

Относительное изменение механической энергии минимально, что означает, что временной шаг был более чем достаточно мал. Шарику потребовалось $1.07\,\text{с}$, чтобы добраться до конца дорожки.

Давайте построим график вычисленных величин!

In [ ]:
t = np.linspace(0, it*dt, it + 1)

# Create figure
fig, axarr = plt.subplots(3, 1, sharex=True, figsize=(10, 8), dpi=400)
fig.subplots_adjust(hspace=0.02)

# Axes 1:
axarr[0].plot(t, x)
axarr[0].set_ylabel(r"$x(t)$, [m]")

# Axes 2: 
axarr[1].plot(t, v)
axarr[1].set_ylabel(r"$v(t)$, [m/s]")

# Axes 2: 
axarr[2].plot(t, U, label="Potential energy, $U$")
axarr[2].plot(t, K, label="Kinetic energy, $K$")
axarr[2].plot(t, K + U, label="Mechanical energy, $E=U+K$")
axarr[2].set_ylabel(r"Energy, [J]")
axarr[2].set_xlabel(r'$t$, [s]')
axarr[2].legend()

plt.show()

Уравнение (2) дает условие для того, когда мяч теряет контакт с дорожкой:

$$g\cos\theta\left(x\right) + v^2\kappa\left(x\right) \leq 0.$$

Давайте проверим, не потерял ли мяч контакт в приведенном выше расчете.

In [ ]:
theta = get_theta(x)
kappa = get_curvature(x)
N = g*np.cos(theta) + v**2*kappa
index = np.argmax(N < 0)
if index <= 0:
    print("The ball did not loose contact.")
else:
    print("The ball lost contact at x = %.2f m."%(x[index]))

Мы также можем проверить, начинает ли мяч скользить или нет. Максимальная статическая сила трения пропорциональна нормальной силе:

\begin{equation}
f_\mathrm{max} = \mu_s N.
\end{equation}

$\mu_s$ называется коэффициентом статического трения. Коэффициент статического трения между пластиком и металлом составляет $0.25<\mu_s<0,4$ [8]. Нормальная сила может быть рассчитана с помощью уравнения (2). Если гравитационное притяжение вдоль дорожки превышает максимальное статическое трение, $mg\sin\theta>f_\mathrm{max}$, мяч начинает скользить.

In [ ]:
mu = 0.25
f = mu*N
Fgx = m*g*np.sin(theta)
index = np.argmax(f < Fgx)
if index <= 1:
    print("The ball did not glide.")
else:
    print("The ball began to glide at x = %.2f m."%(x[index]))

## Анимация

In [ ]:
y = get_y(x)

from matplotlib import animation
from IPython.display import HTML

plt.style.use('default')

# Set up the figure
fig = plt.figure(figsize=(8, 4), dpi=120)
ax = plt.axes(xlim=(xi[0], xi[-1]), 
              ylim=(np.min(yi), np.max(yi)))
ax.set_aspect('equal')

# Define the different elements in the animation
track, = ax.plot(x,y, "k")
obj = plt.Circle((x[0], y[0]), r, fc="grey") # Obj. 1
ax.add_patch(obj)
ax.set_xlabel(r"$x$, [m]")
ax.set_ylabel(r"$y$, [m]")
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
textbox = ax.text(0.775, 0.95, '', transform=ax.transAxes, fontsize=12,
                             verticalalignment='top', bbox=props)

# Calculates the number of frames
FPS = 30
n = len(t)
framesNum = int(FPS*t[-1])

# Animation function. This is called sequentially.
def animate(j):
    i = j*int(n/framesNum)
    theta = get_theta(x[i])
    obj.center = (x[i] + r*np.sin(theta), y[i] + r*np.cos(theta))
    t_str = r'$t = %.2f$ ' % (t[i])
    x_str = r'$x = %.2f$ ' % (x[i])
    y_str = r'$y = %.2f$ ' % (y[i])
    textbox.set_text(t_str + "\n" + x_str + "\n" + y_str)

# Create animation
anim = animation.FuncAnimation(fig, animate, frames=framesNum, interval=1000/FPS)

plt.close(anim._fig)

# Display the animation
HTML(anim.to_html5_video())

## Дальнейшая Работа

- Если мяч будет проскальзывать, дойдет ли он до конца быстрее или медленнее?
- Как мы можем считать, что произошло проскальзывание?

## Resources

[1] NTNU fysikklab: http://home.phys.ntnu.no/brukdef/undervisning/fyslab/. In particular, the assignment text is found at http://home.phys.ntnu.no/brukdef/undervisning/fyslab/files/oppgave2.pdf.

[2] H.D. Young and R.A. Freedman: *University Physics*. Pearson Education, 14th edition, 2016 

[3] J.R. Lien and G. Løvhøiden: *Generell fysikk for universiteter og høgskoler*. Bind 1, Universitetsforlaget, 2001  

[4] R. A. Adams, C. Essex: *Calculus a Complete Course*. Pearson, 8th edition, 2013  

[5] J. Hass, M.D. Weir og G.B. Thomas Jr.: *Thomas’ Calculus*. Pearson Education, 13th edition, 2014  

[6] Wikipedia: *List of moments of inertia*. https://en.wikipedia.org/wiki/List_of_moments_of_inertia. 2017 (aquired 31th November 2017)  

[7] The Engineering ToolBox: *Metals and Alloys - Densities*. https://www.engineeringtoolbox.com/metal-alloys-densities-d_50.html. (aquired 31th November 2017)  

[8] Coefficient of friction: http://www.tribology-abc.com/abc/cof.htm